In [3]:
import numpy as np 
import os
import h5py
import sys
sys.path.insert(0,'/global/common/cori/software/theano/0.8.2/lib/python2.7/site-packages/')
import theano
import theano.tensor as T
sys.path.insert(0,'/global/common/cori/software/keras/1.02/lib/python2.7/site-packages/')
import keras

import numpy as np

from keras.layers import Input, Dense, Convolution2D, MaxPooling2D, UpSampling2D
from keras.models import Model
from keras import regularizers
from keras import optimizers

from keras.callbacks import TensorBoard

import matplotlib.pyplot as plt

def load_data():

    # Load the dataset
    
    # Here we give different data sets for the autoencoders
    dataurl = '/global/homes/s/ssingh79/data/'
    #hdf5file = 'conv_z02.h5'
    hdf5file = 'normalized_data.h5'
    filepath = os.path.join(dataurl, hdf5file)
    
    print("Calling ", hdf5file, "......")
    # Call the load_data method to get back the Final training set. 
    dataset = filepath
    
    with h5py.File(dataset,'r') as hf: 
        #train_set = hf['X_train'][0:1000,0:65536] 
        train_set = hf['data_mean_diff_abs'][0:1000,0:16384]
        print("Printing Train set ", train_set) 
        print("X_train shape ", train_set.shape[0])
        
    return train_set

def build_conv_auto():
    
    input_img = Input(shape=(1, 128, 128))

    x = Convolution2D(16, 3, 3, activation='relu', border_mode='same')(input_img)
    x = MaxPooling2D((2, 2), border_mode='same')(x)
    x = Convolution2D(8, 3, 3, activation='relu', border_mode='same')(x)
    x = MaxPooling2D((2, 2), border_mode='same')(x)
    x = Convolution2D(8, 3, 3, activation='relu', border_mode='same')(x)
    encoded = MaxPooling2D((2, 2), border_mode='same')(x)

    # at this point the representation is (8, 4, 4) i.e. 128-dimensional

    x = Convolution2D(8, 3, 3, activation='relu', border_mode='same')(encoded)
    x = UpSampling2D((2, 2))(x)
    x = Convolution2D(8, 3, 3, activation='relu', border_mode='same')(x)
    x = UpSampling2D((2, 2))(x)
    x = Convolution2D(16, 3, 3, activation='relu')(x)
    x = UpSampling2D((2, 2))(x)
    decoded = Convolution2D(1, 3, 3, activation='sigmoid', border_mode='same')(x)
    
    autoencoder = Model(input_img, decoded)
    autoencoder.compile(optimizer='adadelta', loss='binary_crossentropy')
    
    x_train = load_data()
    
    autoencoder.fit(x_train, x_train,
                nb_epoch=50,
                batch_size=128,
                shuffle=True,
                validation_data=(x_test, x_test),
                callbacks=[TensorBoard(log_dir='/tmp/autoencoder')])
   
    decoded_imgs = autoencoder.predict(x_test)
    
    %matplotlib inline
    plt.subplot(121)
    plt.imshow(x_train[0].reshape(128,128), interpolation = 'None')
    plt.colorbar()
    plt.subplot(122)
    plt.imshow(decoded_imgs[0].reshape(128,128), interpolation = 'None')
    plt.colorbar()
    
    